In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets,grid_search
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold

C:\Users\dim\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\dim\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
newsgroup=datasets.fetch_20newsgroups(subset="all",categories=["sci.med","rec.autos"])

In [3]:
X=newsgroup.data
y=newsgroup.target

In [4]:
vectorizer=TfidfVectorizer()
X=vectorizer.fit_transform(X)

In [5]:
grid={"C":np.power(10.0,np.arange(-5,6))}
cv=KFold(n_splits=5,shuffle=True,random_state=241)
cv=cv.get_n_splits(X)
clf=SVC(kernel="linear",random_state=241)
gs=grid_search.GridSearchCV(clf,grid,scoring="accuracy",cv=cv)
gs.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [6]:
score=0
C=0
for a in gs.grid_scores_:
    if a.mean_validation_score>score:
        score=a.mean_validation_score
        C=a.parameters["C"]

In [7]:
print(score)
print(C)

0.9904040404040404
1.0


In [8]:
print(gs.best_score_)
print(gs.best_params_)

0.9904040404040404
{'C': 1.0}


In [9]:
model=SVC(kernel="linear",random_state=241,C=C)
model.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
words=vectorizer.get_feature_names()
coef=pd.DataFrame(model.coef_.data,model.coef_.indices)
top_words=coef[0].map(lambda w:abs(w)).sort_values(ascending=False).head(10).index.map(lambda i:words[i])
for word in top_words:
    print(word)

car
cars
doctor
of
bmw
information
msg
ford
health
dealer


In [11]:
coef[0].map(lambda w:abs(w)).sort_values(ascending=False).head(10).index.map(lambda i:words[i])

Index(['car', 'cars', 'doctor', 'of', 'bmw', 'information', 'msg', 'ford',
       'health', 'dealer'],
      dtype='object')